# Learning

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [40]:
MODEL = 'ResNet50'
batch_size=8
train_data_path = './CargoContainerImages/train/'
num_classes = 2
max_epoches = 10

In [41]:
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode ='categorical')

Found 147 images belonging to 2 classes.


In [42]:
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input
    model = ResNet50(weights = None, classes=num_classes)
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input
    model = VGG16(weights = None, classes = num_classes)
    output_node_name = 'predictions/Softmax'


In [43]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
19/19 [==============================] - 49s 3s/step - loss: 1.8903 - accuracy: 0.6463
Epoch 2/10
19/19 [==============================] - 47s 2s/step - loss: 0.7126 - accuracy: 0.7823
Epoch 3/10
19/19 [==============================] - 48s 3s/step - loss: 0.1921 - accuracy: 0.9184
Epoch 4/10
19/19 [==============================] - 47s 2s/step - loss: 0.1546 - accuracy: 0.9388
Epoch 5/10
19/19 [==============================] - 48s 3s/step - loss: 0.3270 - accuracy: 0.9320
Epoch 6/10
19/19 [==============================] - 48s 3s/step - loss: 0.3540 - accuracy: 0.9184
Epoch 7/10
19/19 [==============================] - 48s 3s/step - loss: 0.0846 - accuracy: 0.9728
Epoch 8/10
19/19 [==============================] - 48s 3s/step - loss: 0.6353 - accuracy: 0.8503
Epoch 9/10
19/19 [==============================] - 47s 2s/step - loss: 0.3296 - accuracy: 0.9116
Epoch 10/10
19/19 [==============================] - 48s 3s/step - loss: 0.1585 - accuracy: 0.9320


In [44]:
MODEL = 'VGG16'
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
19/19 [==============================] - 48s 3s/step - loss: 0.6193 - accuracy: 0.8299
Epoch 2/10
19/19 [==============================] - 48s 3s/step - loss: 0.4900 - accuracy: 0.9252
Epoch 3/10
19/19 [==============================] - 49s 3s/step - loss: 0.3048 - accuracy: 0.8912
Epoch 4/10
19/19 [==============================] - 48s 3s/step - loss: 0.1373 - accuracy: 0.9524
Epoch 5/10
19/19 [==============================] - 48s 3s/step - loss: 0.0747 - accuracy: 0.9660
Epoch 6/10
19/19 [==============================] - 48s 3s/step - loss: 0.4213 - accuracy: 0.9456
Epoch 7/10
19/19 [==============================] - 48s 3s/step - loss: 0.2752 - accuracy: 0.9048
Epoch 8/10
19/19 [==============================] - 50s 3s/step - loss: 0.4122 - accuracy: 0.9116
Epoch 9/10
19/19 [==============================] - 48s 3s/step - loss: 0.2045 - accuracy: 0.9456
Epoch 10/10
19/19 [==============================] - 48s 3s/step - loss: 0.0977 - accuracy: 0.9660


# Prediction

In [2]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
#import uff

In [15]:
MODEL = 'VGG16'

In [16]:
if MODEL == 'ResNet50':
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    output_node_name  = 'predictions/Softmax'
#sess = K.get_session()
#K.set_learning_phase(0)
#graph = tf.get_default_graph()
model = load_model('saved_model/' + MODEL + 'saved.h5')
model.compile(loss = 'categorical_crossenthropy', optimizer = 'adam', metrics= ['accuracy'])
#output_graph_def = tf.graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ['input_1', output_node_name])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [17]:
#uffmodel = uff.from_tensorflow(output_graph_def, [output_node_name], 
#                              output_filename = 'saved_model/' + MODEL + 'uff')

In [18]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import decode_predictions
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import decode_predictions
def predict_all(folder):
    for filename in os.listdir(folder):
        print(filename)
        fullpath = os.path.join(folder, filename)  # 폴더명과 파일명을 결합
        #print("fullpath", fullpath)  
        try:
            # 테스트할 이미지 불러오기
            img = load_img(fullpath, target_size=(224, 224)) 
            # ResNet에 입력하기 전에 이미지 전처리
            x = img_to_array(img) /255.0
            x = np.expand_dims(x, axis=0)
            #x = preprocess_input(x)

            # 이미지 분류
            preds = model.predict(x)
            print( preds)
            if preds[0][0] > preds[0][1]:
                print('HORIZONTAL')
            else:
                print('VERTICAL')
            #print('Predicted:', decode_predictions(preds, top=1)[0])
            print("")

        except Exception as e:
            print(e)

In [19]:
predict_all('./CargoContainerImages/train/1/')

BHCU2088072.jpg
[[1.0000000e+00 1.8237938e-13]]
HORIZONTAL

CAIU2191783.jpg
[[1.0000000e+00 3.0054383e-12]]
HORIZONTAL

CBCU2000317.jpg
[[1.0000000e+00 1.8323894e-16]]
HORIZONTAL

CCLU1026613.jpg
[[9.9999666e-01 3.3588501e-06]]
HORIZONTAL

CCLU2058301.jpg
[[1.0000000e+00 1.6958339e-09]]
HORIZONTAL

CCLU4755619.jpg
[[1.0000000e+00 1.9567844e-08]]
HORIZONTAL

CCLU4856213.jpg
[[1.000000e+00 8.666594e-14]]
HORIZONTAL

CICU9679964.jpg
[[1.0000000e+00 7.3958994e-10]]
HORIZONTAL

CICU9679964_2.jpg
[[9.999937e-01 6.372226e-06]]
HORIZONTAL

CICU9728204.jpg
[[1.000000e+00 6.753096e-20]]
HORIZONTAL

CISU2000076.jpg
[[1.00000000e+00 1.25731595e-11]]
HORIZONTAL

CISU2000203.jpg
[[1.0000000e+00 2.0447827e-12]]
HORIZONTAL

CKLU2049325.jpg
[[1.00000e+00 6.94793e-12]]
HORIZONTAL

CKLU2049330.jpg
[[1.0000000e+00 8.1068444e-11]]
HORIZONTAL

CKLU4104437.jpg
[[9.9999285e-01 7.1488921e-06]]
HORIZONTAL

CKLU4106701.jpg
[[1.000000e+00 9.833476e-24]]
HORIZONTAL

CKLU4111544.jpg
[[1.000000e+00 9.881908e-17]]
HO

In [20]:
predict_all('./CargoContainerImages/train/2/')

BMOU4699433.jpg
[[0.00321608 0.996784  ]]
VERTICAL

BMOU5782049.jpg
[[8.744585e-05 9.999125e-01]]
VERTICAL

BSIU2575064.jpg
[[0.9700853  0.02991473]]
HORIZONTAL

CAIU2728316.jpg
[[0.18029161 0.8197084 ]]
VERTICAL

CAIU4051570.jpg
[[0.00185946 0.9981406 ]]
VERTICAL

CAIU8121016.jpg
[[0.04309684 0.9569031 ]]
VERTICAL

CARU3858347.jpg
[[9.4777753e-04 9.9905223e-01]]
VERTICAL

CAXU8004650.jpg
[[5.1282923e-04 9.9948716e-01]]
VERTICAL

CBXU2785862.jpg
[[0.01447557 0.9855245 ]]
VERTICAL

CMAU5349828.jpg
[[1.2922657e-05 9.9998713e-01]]
VERTICAL

crop002.jpg
[[0.88725626 0.11274371]]
HORIZONTAL

CRXU1525331.jpg
[[0.00922749 0.9907725 ]]
VERTICAL

DFSU7696722.jpg
[[4.8363995e-04 9.9951637e-01]]
VERTICAL

DFSU7696722_2.jpg
[[0.05789023 0.9421098 ]]
VERTICAL

DFSU7697586.jpg
[[2.9291587e-05 9.9997067e-01]]
VERTICAL

DLSU5000210.JPG
[[0.04027259 0.9597274 ]]
VERTICAL

DLSU5001789.jpg
[[0.9862234  0.01377662]]
HORIZONTAL

DLSU5002635.jpg
[[0.02440058 0.97559947]]
VERTICAL

DLSU5002635_2.jpg
[[0.0331

In [52]:
a = 86
b=61
a/(a+b)

0.5850340136054422

# Extra

In [12]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
 
# imagenet에 미리 훈련된 ResNet50 모델 불러오기
model = ResNet50(weights='imagenet')

 
# 테스트할 이미지 불러오기
img_path = 'test.jpg'
img = load_img(img_path, target_size=(224, 224)) 
 
# ResNet에 입력하기 전에 이미지 전처리
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
 
# 이미지 분류
preds = model.predict(x)
print(preds)
print('Predicted:', decode_predictions(preds, top=3)[0])


[[3.71102676e-10 1.56775593e-09 6.33899333e-07 4.02285906e-07
  1.18995573e-07 3.43044704e-08 1.14061187e-08 1.86256255e-09
  1.97986072e-09 3.11168064e-10 1.82857895e-10 3.71855130e-10
  2.36878317e-09 3.67928488e-09 3.19695798e-10 8.83174811e-10
  3.13072803e-08 3.11225201e-09 3.42527173e-10 3.32528387e-08
  1.96200078e-09 5.57546912e-07 2.18191012e-08 1.51773691e-07
  3.04395904e-08 3.22587219e-08 9.52947854e-09 3.20642490e-09
  2.90862823e-09 5.38676925e-09 4.07433504e-10 1.42372265e-08
  8.15091727e-09 2.41507152e-08 4.86918026e-08 2.22863115e-08
  1.41772318e-08 1.19823573e-09 1.62482561e-09 4.28137570e-09
  4.24195301e-09 1.14656548e-10 2.69182348e-08 1.31458524e-08
  4.61131089e-09 1.36950917e-09 1.47408766e-10 1.41981502e-08
  8.25802760e-10 4.84810414e-09 3.96564914e-09 2.89273453e-06
  4.81442397e-09 1.40846157e-09 1.27366040e-09 4.13008294e-10
  1.33297648e-08 5.33506173e-09 2.46092835e-09 8.66849162e-11
  1.01277942e-09 4.18089989e-08 1.58589213e-08 1.74744330e-10
  6.8164